'''

Predictive Patient Re-admission Model

Date: 14/02/18

Author: Annye Braca

Email: annyebraca@gmail.com

'''

In [252]:
# Importing Libraries
from __future__ import absolute_import, division, print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
from scipy.stats import spearmanr
from pylab import rcParams
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn import feature_extraction
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import model_selection
#import statsmodels.api as sm
import  sys
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import time
import keras
from keras.layers import Flatten, Conv1D, Dense, GlobalAvgPool1D, GlobalMaxPool1D, Dropout, MaxPool1D, BatchNormalization

Populating the interactive namespace from numpy and matplotlib


In [253]:
def set_trace():
    """A Poor mans break point"""
    # without this in iPython debugger can generate strange characters.
    from IPython.core.debugger import Pdb
    Pdb().set_trace(sys._getframe().f_back)


In [254]:
def one_hot_dataframe(data, cols, replace=False):
    """Create one-hot encodings."""
    vec = feature_extraction.DictVectorizer()
    mkdict = lambda row: dict((col, row[col]) for col in cols)
    vecData = pd.DataFrame(vec.fit_transform(data[cols].apply(mkdict, axis=1)).toarray())
    vecData.columns = vec.get_feature_names()
    vecData.index = data.index
    if replace:
        data = data.drop(cols, axis=1)
        data = data.join(vecData)
    return (data, vecData)

In [255]:
# Read in data
data = pd.read_csv('../static/diabetic_data.csv')
discharge = pd.ExcelFile('../static/discharge.xlsx')
discharge = discharge.parse('Data')
admission = pd.ExcelFile('../static/admission.xlsx')
admission = admission.parse('Data')
admission_type = pd.ExcelFile('../static/admission_type_ID.xlsx')
admission_type = admission_type.parse('Data')

In [256]:

data.replace('?' , np.nan, inplace=True)
data.drop(['weight', 'medical_specialty', 'payer_code', 'diag_2', 'diag_3',  'patient_nbr', 'encounter_id'], axis=1, inplace=True)

In [257]:
data = data[~(data['diag_1'].str.startswith('V') | (data['diag_1'].str.startswith('E')))]

In [258]:
# Map discharge and admission id descriptions
discharge_map = dict(zip(discharge['discharge_disposition_id'], discharge['description']))
data['discharge_description'] = data['discharge_disposition_id'].map(discharge_map)
admission_map = dict(zip(admission['admission_source_id'], admission['description']))
data['admission_description'] = data['admission_source_id'].map(admission_map)
# bring in admission type id
admission_type_map = dict(zip(admission_type['admission_type_id'], admission['description']))
data['admission_type_description'] = data['admission_type_id'].map(admission_type_map)
# Drop original ids as these are not dropped in one-hot_encoding function
data.drop(['discharge_disposition_id', 'admission_source_id', 'admission_type_id'], axis=1, inplace=True)

In [259]:

data.dropna(inplace=True)

In [260]:
data, med = one_hot_dataframe(data, ["admission_description",
                                     "discharge_description",
                                     "admission_type_description",
                                     "race",
                                     "gender",
                                     "metformin",
                                     "repaglinide",
                                     "nateglinide",
                                     "chlorpropamide",
                                     "glimepiride",
                                     "acetohexamide",
                                     "glipizide",
                                     "glyburide",
                                     "tolbutamide",
                                     "pioglitazone",
                                     "rosiglitazone",
                                     "acarbose",
                                     "miglitol",
                                     "troglitazone",
                                     "tolazamide",
                                     "examide",
                                     "citoglipton",
                                     "insulin",
                                     "glyburide-metformin",
                                     "glipizide-metformin",
                                     "glimepiride-pioglitazone",
                                     "metformin-rosiglitazone",
                                     "metformin-pioglitazone",
                                     'A1Cresult',
                                     'max_glu_serum',
                                     'change',
                                     'diabetesMed'], replace=True)


In [261]:
inds = pd.isnull(data).any(1).nonzero()[0]
data.ix[inds]

C:\Users\Creative\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app


,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,diag_1,number_diagnoses,...,rosiglitazone=Down,rosiglitazone=No,rosiglitazone=Steady,rosiglitazone=Up,tolazamide=No,tolazamide=Steady,tolbutamide=No,tolbutamide=Steady,troglitazone=No,troglitazone=Steady


In [262]:
# Mapping Age
age_ranges = list(set(data['age']))#
age_ordinal = [1, 8, 7, 6, 0, 9, 4, 2, 5, 3]
age_map = dict(zip(age_ranges, age_ordinal))
data['age'] = data['age'].map(age_map)

In [263]:
#mapping readmitted
readmitted_ranges = list(set(data['readmitted']))
readmitted_ordinal = [2, 1, 0]
readmitted_map = dict(zip(readmitted_ranges, readmitted_ordinal))
data['readmitted'] = data['readmitted'].map(readmitted_map)

In [264]:
###########Subsetting data  , we are interested just in people with a Diabetes condition  #################


In [266]:
diabetes = data[data['diabetesMed=Yes'] == 1]


In [267]:
X = diabetes
inds = pd.isnull(X).any(1).nonzero()[0]
y = diabetes['readmitted']
X.drop('readmitted', axis=1, inplace=True)

C:\Users\Creative\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [268]:
# Separate training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

In [82]:

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.57


In [83]:
kfold = model_selection.KFold(n_splits=10, random_state=7)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

10-fold cross validation average accuracy: 0.564


In [84]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.59      0.86      0.70      8857
          1       0.36      0.02      0.03      1915
          2       0.52      0.33      0.40      6221

avg / total       0.54      0.57      0.51     16993



In [ ]:
###############Convolutional neural networks####################

In [85]:
from keras.regularizers import L1L2

In [270]:
def create_baseline(input_nodes,
                    hidden1_nodes,
                    hidden2_nodes,
                    hidden3_nodes,
                    output_nodes,
                    dropout_rate):
        # NN architecturE
        model = Sequential()
        model.add(GlobalAvgPool1D(input_shape=(146,3,)))
        model.add(Dense(output_dim=hidden1_nodes,
                        input_dim=input_nodes,
                        activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(output_dim=hidden2_nodes,
                        input_dim=hidden1_nodes,
                        activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(output_dim=hidden3_nodes,
                        input_dim=hidden2_nodes,
                        activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(output_dim=hidden4_nodes,
                        input_dim=hidden3_nodes,
                        activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(output_dim=output_nodes,
                        input_dim=hidden4_nodes,
                        activation='sigmoid'))
        # Compile model
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        return model

In [274]:
# Separate training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

In [275]:
# categorical_crossentrophy expects targets to be binary matrices - if target are integer classes
# we need to convert to the expected format using to categorical
# fix random seed for reproducibility
y_train = np.array(y_train)
y_train = [[x] for x in y_train]
y_train = np.array(y_train)
y_test = np.array(y_test)
y_test = [[x] for x in y_test]
y_test = np.array(y_test)

In [167]:
""" Keras Sequential model with Manual validation
"""
start_time = time.time()



y_train = keras.utils.to_categorical(y_train, num_classes=3)
y_test = keras.utils.to_categorical(y_test, num_classes=3)

seed = 13
np.random.seed(seed)
cvscores = []
learning_rate = 0.1
sgd = SGD(lr=learning_rate, momentum=0.05, nesterov=False)

input_nodes = 146
hidden1_nodes = 50
hidden2_nodes = 50
hidden3_nodes = 50
hidden4_nodes = 50
output_nodes = 3
dropout_rate = 0


model = create_baseline(input_nodes,
                        hidden1_nodes,
                        hidden2_nodes,
                        hidden3_nodes,
                        output_nodes,
                        dropout_rate)



model.fit(X_train, y_train, epochs=100, batch_size=128) 
print('--- %s seconds ---'%(time.time() - start_time))
'''
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=model,
                            nb_epoch=100,
                            batch_size=200,
                            verbose=2)
kfold = StratifiedKFold(n_splits=10,
                        shuffle=True,
                        random_state=seed)
results = cross_val_score(estimator,
                          X,
                          y,
                          cv=kfold)
'''
#print("Results of 10-fold Cross-Validation: %.2f%% (%.2f%%)" % (results.mean() * 100, results.std() * 100))

C:\Users\Creative\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=50, activation="relu", input_dim=146)`
C:\Users\Creative\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=50, activation="relu", input_dim=50)`
C:\Users\Creative\Anaconda2\lib\site-packages\ipykernel\__main__.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=50, activation="relu", input_dim=50)`
C:\Users\Creative\Anaconda2\lib\site-packages\ipykernel\__main__.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=50, activation="relu", input_dim=50)`
C:\Users\Creative\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=3, activation="sigmoid", input_dim=50)`


Epoch 1/100
50977/50977 [==============================] - 6s 111us/step - loss: 1.0343 - acc: 0.5132
Epoch 2/100
50977/50977 [==============================] - 5s 97us/step - loss: 0.9502 - acc: 0.5175
Epoch 3/100
50977/50977 [==============================] - 5s 104us/step - loss: 0.9337 - acc: 0.5312
Epoch 4/100
50977/50977 [==============================] - 5s 99us/step - loss: 0.9232 - acc: 0.5528
Epoch 5/100
50977/50977 [==============================] - 5s 100us/step - loss: 0.9186 - acc: 0.5529
Epoch 6/100
50977/50977 [==============================] - 5s 101us/step - loss: 0.9136 - acc: 0.5602
Epoch 7/100
50977/50977 [==============================] - 5s 101us/step - loss: 0.9097 - acc: 0.5585
Epoch 8/100
50977/50977 [==============================] - 5s 100us/step - loss: 0.9075 - acc: 0.5616
Epoch 9/100
50977/50977 [==============================] - 5s 100us/step - loss: 0.9069 - acc: 0.5604
Epoch 10/100
50977/50977 [==============================] - 5s 100us/step - loss: 0.

50977/50977 [==============================] - 5s 100us/step - loss: 0.8660 - acc: 0.5824
Epoch 80/100
50977/50977 [==============================] - 5s 101us/step - loss: 0.8658 - acc: 0.5835
Epoch 81/100
50977/50977 [==============================] - 5s 101us/step - loss: 0.8656 - acc: 0.5827
Epoch 82/100
50977/50977 [==============================] - 5s 100us/step - loss: 0.8646 - acc: 0.5821
Epoch 83/100
50977/50977 [==============================] - 5s 103us/step - loss: 0.8659 - acc: 0.5831
Epoch 84/100
50977/50977 [==============================] - 5s 100us/step - loss: 0.8646 - acc: 0.5838
Epoch 85/100
50977/50977 [==============================] - 5s 100us/step - loss: 0.8630 - acc: 0.5835
Epoch 86/100
50977/50977 [==============================] - 5s 101us/step - loss: 0.8623 - acc: 0.5833
Epoch 87/100
50977/50977 [==============================] - 5s 100us/step - loss: 0.8633 - acc: 0.5839
Epoch 88/100
50977/50977 [==============================] - 5s 104us/step - loss: 0.86

'\n# evaluate model with standardized dataset\nestimator = KerasClassifier(build_fn=model,\n                            nb_epoch=100,\n                            batch_size=200,\n                            verbose=2)\nkfold = StratifiedKFold(n_splits=10,\n                        shuffle=True,\n                        random_state=seed)\nresults = cross_val_score(estimator,\n                          X,\n                          y,\n                          cv=kfold)\n'